# WSI ćwiczenie 3
## Dwuosobowe gry deterministyczne
## Yaroslav Harbar

Celem tego ćwiczenia było zaimplementowanie algorytmu minimax z obcinaniem alpha, beta. Dla poprawnego działania algorytmu stworzyłem funkcje *all_unique_moves*, która zlicza wszystkie możliwe kombinacje liczb, żeby ich suma dawała wynik *aim_value*. W funkcji *heuristic* filtruje te kombinacje tak, aby wszystkie liczby w pojedynczej kombinacji były ze zbioru już wybranych liczb dla bieżącego użytkownika lub ze zbioru jeszcze niewybranych liczb. Następnie szukam wartości zysku bieżącego stanu gry, iterując po każdej kombinacji i licząc ilość liczb w niej występujących, które zostały wybrane przez bieżącego gracza, i sumując ją do ogólnej oceny. Jeżeli gra została ukończona, to funkcja heurystyczna zwraca dużą wartość, w moim przypadku jest to 100000000000000000 (-100000000000000000).

In [2]:
from copy import deepcopy
import random

In [3]:
import itertools
from typing import Iterable, List, Optional
from two_player_games.game import Game
from two_player_games.move import Move
from two_player_games.player import Player
from two_player_games.state import State


class Pick(Game):
    """Class that represents the Pick game"""
    FIRST_PLAYER_DEFAULT_CHAR = '1'
    SECOND_PLAYER_DEFAULT_CHAR = '2'

    def __init__(self, first_player: Player = None, second_player: Player = None, n: int = 4):
        """
        Initializes game.

        Parameters:
            first_player: the player that will go first (if None is passed, a player will be created)
            second_player: the player that will go second (if None is passed, a player will be created)
            n: the subset size of picked numbers that should sum up to aim value
        """

        self.first_player = first_player or Player(self.FIRST_PLAYER_DEFAULT_CHAR)
        self.second_player = second_player or Player(self.SECOND_PLAYER_DEFAULT_CHAR)
        state = PickState(self.first_player, self.second_player, n)
        super().__init__(state)
        self.all_combinations = self.all_unique_moves()

    def all_unique_moves(self):
        candidates = range(1, self.state.max_number + 1)

        def backtrack(i, target):
            combinations = []
            for index in range(i, len(candidates)):
                num = candidates[index]
                if index > i and num == candidates[index - 1]:
                    continue
                if num < target:
                    combinations.extend([num] + comb for comb in backtrack(index + 1, target - num))
                else:
                    if num == target:
                        combinations.append([num])
                    break
            return combinations

        all_combination = backtrack(0, self.state.aim_value)
        all_combination = [item for item in all_combination if len(item) == self.state.n]

        return all_combination


class PickMove(Move):
    """
    Class that represents a move in the PickMove game

    Variables:
        number: selected number (from 1 to n^2)
    """

    def __init__(self, number: int):
        self.number = number

    def __eq__(self, o: object) -> bool:
        if not isinstance(o, PickMove):
            return False
        return self.number == o.number

    def __str__(self) -> str:
        return f"{self.number}"


class PickState(State):
    """Class that represents a state in the PickState game"""

    def __init__(self,
                 current_player: Player, other_player: Player, n,
                 current_player_numbers: List[int] = None,
                 other_player_numbers: List[int] = None):
        """Creates the state. Do not call directly."""

        if current_player_numbers is None:
            current_player_numbers = []
        if other_player_numbers is None:
            other_player_numbers = []

        self.current_player_numbers = current_player_numbers
        self.other_player_numbers = other_player_numbers
        self.selected_numbers = set(self.current_player_numbers).union(self.other_player_numbers)
        self.n = n
        self.max_number = n ** 2
        self.aim_value = int((n ** 2 * (n ** 2 + 1)) / (2 * n))
        super().__init__(current_player, other_player)

    def get_moves(self) -> Iterable[PickMove]:
        return [PickMove(number) for number in range(1, self.max_number + 1) if number not in self.selected_numbers]

    def make_move(self, move: PickMove) -> 'PickState':
        if move.number > self.max_number or move.number in self.selected_numbers:
            raise ValueError("Invalid move")
        else:
            next_player = self._other_player
            next_player_numbers = self.other_player_numbers

            other_player = self._current_player
            other_player_numbers = self.current_player_numbers + [move.number]

        return PickState(
            next_player, other_player, self.n, next_player_numbers, other_player_numbers
        )

    def is_finished(self) -> bool:
        return self._check_if_sums_to_aim_value(self.current_player_numbers) or \
               self._check_if_sums_to_aim_value(self.other_player_numbers) or \
               len(self.selected_numbers) == self.max_number

    def get_winner(self) -> Optional[Player]:
        if not self.is_finished():
            return None
        if self._check_if_sums_to_aim_value(self.current_player_numbers):
            return self._current_player
        elif self._check_if_sums_to_aim_value(self.other_player_numbers):
            return self._other_player
        else:
            return None

    def __str__(self) -> str:
        return f"n: {self.n}, aim_value: {self.aim_value}" \
               f"\nCurrent player: {self._current_player.char}, Numbers: " \
               f"{'[]' if not self.current_player_numbers else sorted(self.current_player_numbers)}," \
               f"\nOther player: {self._other_player.char}, Numbers: " \
               f"{'[]' if not self.other_player_numbers else sorted(self.other_player_numbers)}"

    # below are helper methods for the public interface

    def _check_if_sums_to_aim_value(self, numbers: List[int]) -> bool:
        return self.aim_value in [sum(i) for i in itertools.combinations(numbers, self.n)]

In [4]:
def heuristic(game, maximizing_player):
    if game.is_finished():
        if game.get_winner():
            if maximizing_player:
                return -100000000000000000
            else:
                return 100000000000000000
        else:
            return 0

    current_player_moves = game.state.current_player_numbers
    possible_moves = [item.number for item in game.state.get_moves()]

    new_combinations = []
    score = 0

    for combination in game.all_combinations:
        good = True
        for item in combination:
            if item not in possible_moves and item not in current_player_moves:
                good = False
                break

        if good:
            new_combinations.append(combination)

    for combination in new_combinations:
        count = 0
        for item in combination:
            if item in current_player_moves:
                count += 1
        score += pow(10, count)

    if maximizing_player:
        return -score
    return score

## Implementacja algorytmu minmax z obcinaniem a, b
Algorytm polega na minimalizowaniu (maksymalizowaniu) możliwego zysku oraz redukcję stanów, w przypadku gdy algorytm znalazł przynajmniej jedno rozwiązanie czyniące obecnie badaną opcję gorszą od poprzednio zbadanych opcji. W środku tego algorytmu zaimplementowałem możliwość zwracania losowego ruchu spośród ruchów, który dają ten sam maksymalny (minimalny) wynik.

In [5]:
def minimax(game: Pick, depth, alpha, beta, maximizing_player):
    if depth == 0 or game.state.is_finished():
        return [heuristic(game, maximizing_player), 0]

    if maximizing_player:
        max_val = float('-inf')
        moves = {}

        for possible_move in game.get_moves():
            game_copy = deepcopy(game)
            game_copy.make_move(possible_move)

            [val, _] = minimax(game_copy, depth - 1, alpha, beta, False)

            if val > max_val:
                max_val = val
                moves = {f"{max_val}": [possible_move]}
            elif val == max_val:
                moves[f"{max_val}"].append(possible_move)

            alpha = max(alpha, max_val)

            if beta <= alpha:
                break
        return [max_val, random.choice(moves[f"{max_val}"])]
    else:
        min_val = float('inf')
        moves = {}

        for possible_move in game.get_moves():
            game_copy = deepcopy(game)
            game_copy.make_move(possible_move)

            [val, _] = minimax(game_copy, depth - 1, alpha, beta, True)

            if val < min_val:
                min_val = val
                moves = {f"{min_val}": [possible_move]}
            elif val == min_val:
                moves[f"{min_val}"].append(possible_move)

            beta = min(beta, min_val)

            if beta <= alpha:
                break
        return [min_val, random.choice(moves[f"{min_val}"])]

Stworzyłem też funkcje *two_computers_game* do symulacji gry dwóch komputerów, dzięki której testuję działanie algorytmu.

In [6]:
def two_computers_game(size, depth_arr, p=False):
    game = Pick(n=size)
    player = 0

    while not game.state.is_finished():
        [combinations, move] = minimax(game, depth_arr[player], float('-inf'), float('inf'), True)

        if p:
            print(f"player: {player+1}, scale value: {combinations}, move: {move.number}, PLAYER_MOVES: {game.state.current_player_numbers}")

        game.make_move(move)

        player = 0 if player == 1 else 1

    if p:
        if game.state.get_winner():
            print("Winner: ", game.state.get_winner().char)
        else:
            print("Draw")

    # return game.state.get_winner()

# Symulacja gry dwóch komputerów
Przeprowadziłem symulacje dla różnych wielkości gry oraz dla różnych głębokości AI_1 oraz AI_2.

In [7]:
two_computers_game(4, [1,1], p=True)

player: 1, scale value: 67, move: 1, PLAYER_MOVES: []
player: 2, scale value: 228, move: 2, PLAYER_MOVES: []
player: 1, scale value: 200, move: 4, PLAYER_MOVES: [1]
player: 2, scale value: 515, move: 3, PLAYER_MOVES: [2]
player: 1, scale value: 445, move: 5, PLAYER_MOVES: [1, 4]
player: 2, scale value: 1031, move: 7, PLAYER_MOVES: [2, 3]
player: 1, scale value: 1100, move: 6, PLAYER_MOVES: [1, 4, 5]
player: 2, scale value: 1740, move: 8, PLAYER_MOVES: [2, 3, 7]
player: 1, scale value: 2410, move: 12, PLAYER_MOVES: [1, 4, 5, 6]
player: 2, scale value: 100000000000000000, move: 16, PLAYER_MOVES: [2, 3, 7, 8]
Winner:  2


In [8]:
two_computers_game(4, [2,1], p=True)

player: 1, scale value: -228, move: 16, PLAYER_MOVES: []
player: 2, scale value: 228, move: 15, PLAYER_MOVES: []
player: 1, scale value: -416, move: 14, PLAYER_MOVES: [16]
player: 2, scale value: 416, move: 12, PLAYER_MOVES: [15]
player: 1, scale value: -852, move: 13, PLAYER_MOVES: [16, 14]
player: 2, scale value: 852, move: 8, PLAYER_MOVES: [15, 12]
player: 1, scale value: -1390, move: 7, PLAYER_MOVES: [16, 14, 13]
player: 2, scale value: 1390, move: 11, PLAYER_MOVES: [15, 12, 8]
player: 1, scale value: -4830, move: 3, PLAYER_MOVES: [16, 14, 13, 7]
player: 2, scale value: 4830, move: 6, PLAYER_MOVES: [15, 12, 8, 11]
player: 1, scale value: 100000000000000000, move: 1, PLAYER_MOVES: [16, 14, 13, 7, 3]
Winner:  1


In [9]:
two_computers_game(4, [3,1], p=True)

player: 1, scale value: 173, move: 10, PLAYER_MOVES: []
player: 2, scale value: 247, move: 16, PLAYER_MOVES: []
player: 1, scale value: 338, move: 15, PLAYER_MOVES: [10]
player: 2, scale value: 668, move: 13, PLAYER_MOVES: [16]
player: 1, scale value: 1797, move: 5, PLAYER_MOVES: [10, 15]
player: 2, scale value: 1977, move: 9, PLAYER_MOVES: [16, 13]
player: 1, scale value: 100000000000000000, move: 3, PLAYER_MOVES: [10, 15, 5]
player: 2, scale value: 4013, move: 1, PLAYER_MOVES: [16, 13, 9]
player: 1, scale value: 100000000000000000, move: 4, PLAYER_MOVES: [10, 15, 5, 3]
Winner:  1


In [10]:
two_computers_game(4, [4,1], p=True)

player: 1, scale value: -416, move: 1, PLAYER_MOVES: []
player: 2, scale value: 228, move: 2, PLAYER_MOVES: []
player: 1, scale value: -852, move: 3, PLAYER_MOVES: [1]
player: 2, scale value: 416, move: 5, PLAYER_MOVES: [2]
player: 1, scale value: -2187, move: 16, PLAYER_MOVES: [1, 3]
player: 2, scale value: 1883, move: 15, PLAYER_MOVES: [2, 5]
player: 1, scale value: 100000000000000000, move: 14, PLAYER_MOVES: [1, 3, 16]
Winner:  1


In [11]:
two_computers_game(4, [1,2], p=True)

player: 1, scale value: 67, move: 16, PLAYER_MOVES: []
player: 2, scale value: -161, move: 1, PLAYER_MOVES: []
player: 1, scale value: 161, move: 2, PLAYER_MOVES: [16]
player: 2, scale value: -245, move: 3, PLAYER_MOVES: [1]
player: 1, scale value: 245, move: 5, PLAYER_MOVES: [16, 2]
player: 2, scale value: -853, move: 11, PLAYER_MOVES: [1, 3]
player: 1, scale value: 853, move: 4, PLAYER_MOVES: [16, 2, 5]
player: 2, scale value: -100000000000000000, move: 7, PLAYER_MOVES: [1, 3, 11]
player: 1, scale value: 100000000000000000, move: 12, PLAYER_MOVES: [16, 2, 5, 4]
Winner:  1


In [12]:
two_computers_game(4, [2,2], p=True)

player: 1, scale value: -228, move: 1, PLAYER_MOVES: []
player: 2, scale value: -161, move: 16, PLAYER_MOVES: []
player: 1, scale value: -277, move: 2, PLAYER_MOVES: [1]
player: 2, scale value: -173, move: 15, PLAYER_MOVES: [16]
player: 1, scale value: -253, move: 3, PLAYER_MOVES: [1, 2]
player: 2, scale value: -131, move: 14, PLAYER_MOVES: [16, 15]
player: 1, scale value: -185, move: 5, PLAYER_MOVES: [1, 2, 3]
player: 2, scale value: -84, move: 12, PLAYER_MOVES: [16, 15, 14]
player: 1, scale value: -101, move: 4, PLAYER_MOVES: [1, 2, 3, 5]
player: 2, scale value: -21, move: 13, PLAYER_MOVES: [16, 15, 14, 12]
player: 1, scale value: -30, move: 7, PLAYER_MOVES: [1, 2, 3, 5, 4]
player: 2, scale value: 0, move: 11, PLAYER_MOVES: [16, 15, 14, 12, 13]
player: 1, scale value: 0, move: 6, PLAYER_MOVES: [1, 2, 3, 5, 4, 7]
player: 2, scale value: 0, move: 9, PLAYER_MOVES: [16, 15, 14, 12, 13, 11]
player: 1, scale value: 0, move: 10, PLAYER_MOVES: [1, 2, 3, 5, 4, 7, 6]
player: 2, scale value: 0,

In [13]:
two_computers_game(4, [3,2], p=True)

player: 1, scale value: 173, move: 8, PLAYER_MOVES: []
player: 2, scale value: -166, move: 13, PLAYER_MOVES: []
player: 1, scale value: 429, move: 15, PLAYER_MOVES: [8]
player: 2, scale value: -429, move: 16, PLAYER_MOVES: [13]
player: 1, scale value: 1806, move: 7, PLAYER_MOVES: [8, 15]
player: 2, scale value: -1806, move: 4, PLAYER_MOVES: [13, 16]
player: 1, scale value: 100000000000000000, move: 1, PLAYER_MOVES: [8, 15, 7]
player: 2, scale value: -100000000000000000, move: 3, PLAYER_MOVES: [13, 16, 4]
player: 1, scale value: 100000000000000000, move: 11, PLAYER_MOVES: [8, 15, 7, 1]
Winner:  1


In [14]:
two_computers_game(4, [4,2], p=True)

player: 1, scale value: -416, move: 11, PLAYER_MOVES: []
player: 2, scale value: -165, move: 14, PLAYER_MOVES: []
player: 1, scale value: -1009, move: 1, PLAYER_MOVES: [11]
player: 2, scale value: -226, move: 16, PLAYER_MOVES: [14]
player: 1, scale value: -905, move: 7, PLAYER_MOVES: [11, 1]
player: 2, scale value: -293, move: 15, PLAYER_MOVES: [14, 16]
player: 1, scale value: 100000000000000000, move: 12, PLAYER_MOVES: [11, 1, 7]
player: 2, scale value: -100000000000000000, move: 9, PLAYER_MOVES: [14, 16, 15]
player: 1, scale value: 100000000000000000, move: 8, PLAYER_MOVES: [11, 1, 7, 12]
player: 2, scale value: -100000000000000000, move: 13, PLAYER_MOVES: [14, 16, 15, 9]
player: 1, scale value: 100000000000000000, move: 2, PLAYER_MOVES: [11, 1, 7, 12, 8]
player: 2, scale value: -100000000000000000, move: 5, PLAYER_MOVES: [14, 16, 15, 9, 13]
player: 1, scale value: 100000000000000000, move: 3, PLAYER_MOVES: [11, 1, 7, 12, 8, 2]
Winner:  1


In [26]:
two_computers_game(4, [1,3], p=True)

player: 1, scale value: 67, move: 16, PLAYER_MOVES: []
player: 2, scale value: 416, move: 15, PLAYER_MOVES: []
player: 1, scale value: 200, move: 14, PLAYER_MOVES: [16]
player: 2, scale value: 852, move: 8, PLAYER_MOVES: [15]
player: 1, scale value: 596, move: 3, PLAYER_MOVES: [16, 14]
player: 2, scale value: 2852, move: 1, PLAYER_MOVES: [15, 8]
player: 1, scale value: 1721, move: 2, PLAYER_MOVES: [16, 14, 3]
player: 2, scale value: 100000000000000000, move: 10, PLAYER_MOVES: [15, 8, 1]
Winner:  2


In [16]:
two_computers_game(4, [2,3], p=True)

player: 1, scale value: -228, move: 16, PLAYER_MOVES: []
player: 2, scale value: 416, move: 15, PLAYER_MOVES: []
player: 1, scale value: -416, move: 14, PLAYER_MOVES: [16]
player: 2, scale value: 852, move: 8, PLAYER_MOVES: [15]
player: 1, scale value: -852, move: 13, PLAYER_MOVES: [16, 14]
player: 2, scale value: 3742, move: 10, PLAYER_MOVES: [15, 8]
player: 1, scale value: -3742, move: 1, PLAYER_MOVES: [16, 14, 13]
player: 2, scale value: -100000000000000000, move: 11, PLAYER_MOVES: [15, 8, 10]
player: 1, scale value: 100000000000000000, move: 6, PLAYER_MOVES: [16, 14, 13, 1]
Winner:  1


In [17]:
two_computers_game(4, [3,3], p=True)

player: 1, scale value: 173, move: 7, PLAYER_MOVES: []
player: 2, scale value: 557, move: 2, PLAYER_MOVES: []
player: 1, scale value: 427, move: 1, PLAYER_MOVES: [7]
player: 2, scale value: 1092, move: 3, PLAYER_MOVES: [2]
player: 1, scale value: 1637, move: 11, PLAYER_MOVES: [7, 1]
player: 2, scale value: 3045, move: 15, PLAYER_MOVES: [2, 3]
player: 1, scale value: 100000000000000000, move: 16, PLAYER_MOVES: [7, 1, 11]
player: 2, scale value: 100000000000000000, move: 14, PLAYER_MOVES: [2, 3, 15]
Winner:  2


In [23]:
two_computers_game(4, [4,3], p=True)

player: 1, scale value: -416, move: 9, PLAYER_MOVES: []
player: 2, scale value: 557, move: 15, PLAYER_MOVES: []
player: 1, scale value: -938, move: 2, PLAYER_MOVES: [9]
player: 2, scale value: 938, move: 1, PLAYER_MOVES: [15]
player: 1, scale value: -1263, move: 7, PLAYER_MOVES: [9, 2]
player: 2, scale value: 1263, move: 16, PLAYER_MOVES: [15, 1]
player: 1, scale value: 100000000000000000, move: 12, PLAYER_MOVES: [9, 2, 7]
player: 2, scale value: -100000000000000000, move: 8, PLAYER_MOVES: [15, 1, 16]
player: 1, scale value: 100000000000000000, move: 10, PLAYER_MOVES: [9, 2, 7, 12]
player: 2, scale value: -100000000000000000, move: 11, PLAYER_MOVES: [15, 1, 16, 8]
player: 1, scale value: 100000000000000000, move: 6, PLAYER_MOVES: [9, 2, 7, 12, 10]
Winner:  1


In [19]:
two_computers_game(4, [1,4], p=True)

player: 1, scale value: 67, move: 16, PLAYER_MOVES: []
player: 2, scale value: -245, move: 1, PLAYER_MOVES: []
player: 1, scale value: 161, move: 2, PLAYER_MOVES: [16]
player: 2, scale value: -853, move: 5, PLAYER_MOVES: [1]
player: 1, scale value: 345, move: 3, PLAYER_MOVES: [16, 2]
player: 2, scale value: -1964, move: 13, PLAYER_MOVES: [1, 5]
player: 1, scale value: 1655, move: 7, PLAYER_MOVES: [16, 2, 3]
player: 2, scale value: 100000000000000000, move: 15, PLAYER_MOVES: [1, 5, 13]
Winner:  2


In [20]:
two_computers_game(4, [2,4], p=True)

player: 1, scale value: -228, move: 16, PLAYER_MOVES: []
player: 2, scale value: -245, move: 1, PLAYER_MOVES: []
player: 1, scale value: -277, move: 15, PLAYER_MOVES: [16]
player: 2, scale value: -225, move: 2, PLAYER_MOVES: [1]
player: 1, scale value: -253, move: 14, PLAYER_MOVES: [16, 15]
player: 2, scale value: -161, move: 3, PLAYER_MOVES: [1, 2]
player: 1, scale value: -185, move: 12, PLAYER_MOVES: [16, 15, 14]
player: 2, scale value: -160, move: 6, PLAYER_MOVES: [1, 2, 3]
player: 1, scale value: -211, move: 13, PLAYER_MOVES: [16, 15, 14, 12]
player: 2, scale value: -110, move: 9, PLAYER_MOVES: [1, 2, 3, 6]
player: 1, scale value: -330, move: 11, PLAYER_MOVES: [16, 15, 14, 12, 13]
player: 2, scale value: 0, move: 7, PLAYER_MOVES: [1, 2, 3, 6, 9]
player: 1, scale value: -200, move: 8, PLAYER_MOVES: [16, 15, 14, 12, 13, 11]
player: 2, scale value: 0, move: 4, PLAYER_MOVES: [1, 2, 3, 6, 9, 7]
player: 1, scale value: 0, move: 10, PLAYER_MOVES: [16, 15, 14, 12, 13, 11, 8]
player: 2, sca

In [21]:
two_computers_game(4, [3,4], p=True)

player: 1, scale value: 173, move: 10, PLAYER_MOVES: []
player: 2, scale value: -330, move: 1, PLAYER_MOVES: []
player: 1, scale value: 330, move: 4, PLAYER_MOVES: [10]
player: 2, scale value: -827, move: 13, PLAYER_MOVES: [1]
player: 1, scale value: 827, move: 16, PLAYER_MOVES: [10, 4]
player: 2, scale value: -772, move: 9, PLAYER_MOVES: [1, 13]
player: 1, scale value: 772, move: 11, PLAYER_MOVES: [10, 4, 16]
player: 2, scale value: -100000000000000000, move: 8, PLAYER_MOVES: [1, 13, 9]
player: 1, scale value: 100000000000000000, move: 3, PLAYER_MOVES: [10, 4, 16, 11]
Winner:  1


In [22]:
two_computers_game(4, [4,4], p=True)

player: 1, scale value: -416, move: 1, PLAYER_MOVES: []
player: 2, scale value: -245, move: 16, PLAYER_MOVES: []
player: 1, scale value: -505, move: 2, PLAYER_MOVES: [1]
player: 2, scale value: -225, move: 15, PLAYER_MOVES: [16]
player: 1, scale value: -431, move: 3, PLAYER_MOVES: [1, 2]
player: 2, scale value: -161, move: 14, PLAYER_MOVES: [16, 15]
player: 1, scale value: -330, move: 4, PLAYER_MOVES: [1, 2, 3]
player: 2, scale value: -80, move: 13, PLAYER_MOVES: [16, 15, 14]
player: 1, scale value: -240, move: 5, PLAYER_MOVES: [1, 2, 3, 4]
player: 2, scale value: -10, move: 12, PLAYER_MOVES: [16, 15, 14, 13]
player: 1, scale value: -100, move: 11, PLAYER_MOVES: [1, 2, 3, 4, 5]
player: 2, scale value: 0, move: 8, PLAYER_MOVES: [16, 15, 14, 13, 12]
player: 1, scale value: 0, move: 6, PLAYER_MOVES: [1, 2, 3, 4, 5, 11]
player: 2, scale value: 0, move: 10, PLAYER_MOVES: [16, 15, 14, 13, 12, 8]
player: 1, scale value: 0, move: 7, PLAYER_MOVES: [1, 2, 3, 4, 5, 11, 6]
player: 2, scale value: 

| First Computer Depth | Second Computer Depth | Result     |
|----------------------|-----------------------|------------|
| 1                    | 1                     | Computer 2 |
| 2                    | 1                     | Computer 1 |
| 3                    | 1                     | Computer 1 |
| 4                    | 1                     | Computer 1 |
| 1                    | 2                     | Computer 1 |
| 2                    | 2                     | Draw       |
| 3                    | 2                     | Computer 1 |
| 4                    | 2                     | Computer 1 |
| 1                    | 3                     | Computer 2 |
| 2                    | 3                     | Computer 1 |
| 3                    | 3                     | Computer 2 |
| 4                    | 3                     | Computer 1 |
| 1                    | 4                     | Computer 2 |
| 2                    | 4                     | Draw       |
| 3                    | 4                     | Computer 1 |
| 4                    | 4                     | Draw       |

# Gra człowieka z komputerem
Program jest zaimplementowany w pliku **lab3.py**.

# Wnioski

Analizując powyższe wyniki, można wywnioskować, że od tego, kto zwycięży, zależy głębokość przeszukiwania. Z im wyższą głębokością mamy do czynienia, tym większe jest prawdopodobieństwo, że bieżący gracz zwycięży, jednakże są przypadki, kiedy wygrywał komputer, który miał mniejszą głębokość. Zauważyłem też, że gracz, który zaczyna pierwszy posiada przewagę nad drugim graczem. Dobrze to widać, kiedy podajemy te same głębokości dla obu komputerów, gdzie dla większości przypadków zwycięży pierwszy gracz.
Dobierając głębokość, trzeba uważać na jej wartość, ponieważ im większa jest ta wartość, tym wolniej działa algorytm, ale daje zdecydowanie lepszy wynik.